<a href="https://colab.research.google.com/github/gihoonpark/Security-Threat-Detection-Anomaly-Detection-Time-series/blob/main/%EC%82%B0%EC%97%85%EC%A0%9C%EC%96%B4_%EB%B3%B4%EC%95%88AI_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Library


In [ ]:
import sys

from pathlib import Path
from datetime import timedelta
import dateutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import trange
import pandas as pd
import numpy as np
import seaborn as sns

Load Data

In [ ]:
train1 = pd.read_csv('/content/drive/My Drive/dataset/산업보안AI_data/HAI 2.0/training/train1.csv')
train2 = pd.read_csv('/content/drive/My Drive/dataset/산업보안AI_data/HAI 2.0/training/train2.csv')
train3 = pd.read_csv('/content/drive/My Drive/dataset/산업보안AI_data/HAI 2.0/training/train3.csv')

TRAIN_DF_RAW = pd.concat([train1, train2, train3], axis=0)
TRAIN_DF_RAW

In [ ]:
TIMESTAMP_FIELD = "time"
IDSTAMP_FIELD = 'id'
ATTACK_FIELD = "attack"
VALID_COLUMNS_IN_TRAIN_DATASET = TRAIN_DF_RAW.columns.drop([TIMESTAMP_FIELD])
VALID_COLUMNS_IN_TRAIN_DATASET

In [ ]:
sns.heatmap(TRAIN_DF_RAW.corr())

Preprocessing

In [ ]:
# train_dataset field별 min, maxr값
TAG_MIN = TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].min()
TAG_MAX = TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].max() 

In [ ]:
def normalize(df):
    ndf = df.copy()
    for c in df.columns:
        if TAG_MIN[c] == TAG_MAX[c]: # 최소, 최대값이 같은 필드의 값들은 모두 0으로 만듦
            ndf[c] = df[c] - TAG_MIN[c]
        else:
            ndf[c] = (df[c] - TAG_MIN[c]) / (TAG_MAX[c] - TAG_MIN[c]) # 위 경우 외 정규화
    return ndf

In [ ]:
TRAIN_DF = normalize(TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET])
np.array(TRAIN_DF) #(921603, 80)

In [ ]:
'''
TIME_STEPS = 90

def create_sequences(values, time_steps=TIME_STEPS):
    output = []
    for i in range(len(values) - time_steps):
        output.append(values[i : (i + time_steps)])
    # Convert 2D sequences into 3D as we will be feeding this into
    # a convolutional layer.
    return np.expand_dims(output, axis=2) # cnn model로 들어갈 input이기 때문에 차원 추가.

x_train = create_sequences(TRAIN_DF)
print("Training input shape: ", x_train.shape)
'''

In [ ]:
# 시계열 모형 input에 적합하도록 전처리(window, pad sequence)

WINDOW_GIVEN = 89
WINDOW_SIZE = 90

class HaiDataset():
    def __init__(self, timestamps, df, stride=10, attacks=None):
        self.ts = np.array(timestamps)
        self.tag_values = np.array(df, dtype=np.float32)
        self.valid_idxs = []
        for L in trange(len(self.ts) - WINDOW_SIZE + 1): # trange(921603-90+1 = 921514) , L = 0,1,2,...,921513
            R = L + WINDOW_SIZE - 1 # R = 89,90,91,...,921602
            if dateutil.parser.parse(self.ts[R]) - dateutil.parser.parse( 
                self.ts[L] # parse('2016-04-16')-> datetime.datetime(2016, 4, 16, 0, 0) , 90번째 timestap 마다 같은지 확인
                # dateutil.parser.parse(self.ts[R]): 89,90,..921602 번째 timestap을 datetime형 데이터로 바꾸고,
                # dateutil.parser.parse(self.ts[L]): 0,1,2,...921513번째 timestap을 datetime형 데이터로 바꿔서 이 둘 차이가 만약 timedelta(89초)이면 if문 실행
            ) == timedelta(seconds=WINDOW_SIZE - 1): # timedelta : 두 날짜/시간의 차이인 기간.
                self.valid_idxs.append(L) # 90번째 timstap가 될때마다 timestap의 인덱스 L을 valid_idxs에 추가. valid_idxs = (L = 0,1,2,...921513 )
        self.valid_idxs = np.array(self.valid_idxs, dtype=np.int32)[::stride] # valid_idxs=(0,1,2,...921513)를 10번째씩 추출, valid_idxs 재구성, 92134개 생김
        self.n_idxs = len(self.valid_idxs) # n_idxs = 92134 , valid_idxs=[0  10  20 ... 921488 921498 921508] -> 아마 데이터가 항상 1초 간격은 아님을 확인
        print(f"# of valid windows: {self.n_idxs}") # n_idxs = 92134
        if attacks is not None: # 현재는 attacks = None이므로 HaiDataset class의 객체인 HAI_DATASET_TRAIN.with_attacks는 False이다.
            self.attacks = np.array(attacks, dtype=np.float32)
            self.with_attack = True
        else:
            self.with_attack = False

    def __len__(self):
        return self.n_idxs

    def __getitem__(self, idx):
        i = self.valid_idxs[idx] #  idx로 인덱싱 가능 ///  idx=0 이면, i=0. idx=1 이면, i=10. ...  (valid_idxs=(0,1,2,...921513))
        last = i + WINDOW_SIZE - 1 # last = [89, 99, 109, ...]
        item = {"attack": self.attacks[last]} if self.with_attack else {}
        item["ts"] = self.ts[i + WINDOW_SIZE - 1] # timestap ts : [89, 99, 109, ...] 데이터들 , 2020-09-11-00-01-29 , 2020-09-11-00-01-39, ...
        item["given"] = torch.from_numpy(self.tag_values[i : i + WINDOW_GIVEN]) 
        # torch.from_numpy :numpy를 tensor 자료형으로 바꿈, df데이터의 0~88, 10~98, 20~108, ...,921508~921596 index를 가지는 input 데이터 (시계열데이터수92134,timestap89,다변량80) 
        item["answer"] = torch.from_numpy(self.tag_values[last]) # 0~88, 10~98, 20~108... given 시계열 데이터로 89, 99, 109, ...시계열데이터 예측하기전, 정답 output 데이터.
        return item


In [ ]:
HAI_DATASET_TRAIN = HaiDataset(TRAIN_DF_RAW[TIMESTAMP_FIELD], TRAIN_DF, stride=10)

Modeling

In [ ]:
N_HIDDENS = 100
N_LAYERS = 3
BATCH_SIZE = 512

class StackedGRU(torch.nn.Module):
    def __init__(self, n_tags):  # layer 정의.
        super().__init__() 
        # stackedGRU 클래스는 torch.nn.Module 클래스를 상속받음
        # 먼저 stackedGRU의 n_tag 속성을 부르고, super().__init__통해 torch.nn.Module 의 __init__ 호출.
        self.rnn = torch.nn.GRU(
            input_size=n_tags, # n_tags = 80
            hidden_size=N_HIDDENS,
            num_layers=N_LAYERS,
            bidirectional=True,
            dropout=0, # batch_first를 True로 안하면 time-step(=sequence_length), batch_size, input_vector 형태.
        )
        self.fc = torch.nn.Linear(N_HIDDENS * 2, n_tags) # N_HIDDENS * 2의 2는 bidirectional_lstm때문, # output_tensor : (Batch, seq=89, n_tags = 80)

    def forward(self, x):  # 위에서 정의한 layer로 데이터 x 정의, foward 훈련 과정 정의.
        x = x.transpose(0, 1)  # 기존 x : (batch, seq, params) -> 변화 x : (seq, batch, params), 왜할까? -> 위에서 batch_first = True로 안했기 때문.
        self.rnn.flatten_parameters() # cell간 compact해야해서 호출? -? 개인적으로 공부 필요
        # RNN 셀은 두 개의 입력을 리턴, 첫번째 리턴값은 모든 시점(timesteps)의 은닉 상태, output : seq_output
        # 두번째 리턴값은 마지막 시점(timestep)의 은닉 상태, hidden_state_output : 
        # 요약 : (output, (hidden or hidden,cell)) 의 tuple 형태, LSTM만 cell state있음
        outs, _ = self.rnn(x)
        out = self.fc(outs[-1]) # many to one이므로 seq_output 중 가장 마지막 seq이 필요. 
        return x[0] + out # 왜 x[0]??? -> 각 배치 데이터의 첫번째 seq (batch_size, n_tage) 데이터들을 rnn output값에 더해서 skip-connection 구현.
        # 단순히 rnn ouptut만 도출해도 무방함.

In [ ]:
MODEL = StackedGRU(n_tags=TRAIN_DF.shape[1]) # TRAIN_DF.shape[1] = 80
MODEL.cuda()

Training

In [ ]:
def train(dataset, model, batch_size, n_epochs):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True) #HAI_DATASET_TRAIN호출, 배치셋 만듦
    optimizer = torch.optim.AdamW(model.parameters()) # optimizer는 adamW로 설정.
    loss_fn = torch.nn.MSELoss() # 손실함수는 MSE로 설정.
    epochs = trange(n_epochs, desc="training") # epochs = [0,1,2,3,4,... n_epochs]
    best = {"loss": sys.float_info.max}
    loss_history = []
    for e in epochs:
        epoch_loss = 0
        for batch in dataloader:
            optimizer.zero_grad() # 초기화, backward()를 호출할 때마다 변화도가 버퍼(buffer)에 누적되기 때문.
            given = batch["given"].cuda() # 배치셋의 given 데이터를 x = given으로 정의 
            guess = model(given) # MODEL에 x 대입 -> (x:given) -> y_hat 도출 (y_hat = guess)
            answer = batch["answer"].cuda() # 배치셋의 answer 데이터를 y = given으로 정의 
            loss = loss_fn(answer, guess) # answer과 guess 간 손실함수 대입해서 loss 값 구함
            loss.backward() # 역전파 단계: 모델의 매개변수에 대한 손실의 변화도를 계산.
            epoch_loss += loss.item() # loss는 (1,) 형태의 Tensor이며, loss.item()은 loss의 스칼라 값
            optimizer.step() # Optimizer의 step 함수를 호출하면 매개변수가 갱신

        loss_history.append(epoch_loss) # loss_history : 배치별 loss 값의 합을 loss_history에 저장.
        epochs.set_postfix_str(f"loss: {epoch_loss:.6f}")
        if epoch_loss < best["loss"]: #  가장 작은 loss를 찾을때까지 계속 best 사전의 loss item값을 epcoh마다 업데이트
            best["state"] = model.state_dict() 
            # torch.nn.Module 모델의 학습 가능한 매개변수(예. 가중치와 편향)들은 모델의 매개변수에 포함되어 있다 (model.parameters()로 접근)
            # state_dict 는 간단히 말하면 각 계층을 매개변수 텐서로 매핑되는 Python 사전(dict) 객체
            best["loss"] = epoch_loss
            best["epoch"] = e + 1
    return best, loss_history

In [ ]:
%%time
MODEL.train()
BEST_MODEL, LOSS_HISTORY = train(HAI_DATASET_TRAIN, MODEL, BATCH_SIZE, 32)

In [ ]:
BEST_MODEL["loss"], BEST_MODEL["epoch"]

In [ ]:
import os
print(os.getcwd())

Save and load model

In [ ]:
 torch.save({
     "state": BEST_MODEL["state"],
     "best_epoch": BEST_MODEL["epoch"],
     "loss_history": LOSS_HISTORY,
     }, '/content/drive/MyDrive/dataset/산업보안AI_data/' + 'model.pt')
# 전체 모델을 저장하거나, 모델의 state_dict를 저장 할 때 사용.

In [ ]:
SAVED_MODEL = torch.load('/content/drive/MyDrive/dataset/산업보안AI_data/' + 'model.pt') 
# 전체 모델을 불러오거나, 모델의 state_dict를 불러 올 때 사용

MODEL.load_state_dict(SAVED_MODEL["state"]) 
# state_dict를 이용하여, 모델 객체 내의 매개 변수 값을 초기화.
# 모델을 불러 온 이후에는 이 모델을 학습 할 껀지, 사용 할 껀지에 따라 각각 model.train(), model.eval() 둘 중에 하나를 사용

Test

In [ ]:
test1 = pd.read_csv('/content/drive/My Drive/dataset/산업보안AI_data/HAI 2.0/testing/test1.csv')
test2 = pd.read_csv('/content/drive/My Drive/dataset/산업보안AI_data/HAI 2.0/testing/test2.csv')
test3= pd.read_csv('/content/drive/My Drive/dataset/산업보안AI_data/HAI 2.0/testing/test3.csv')
test4= pd.read_csv('/content/drive/My Drive/dataset/산업보안AI_data/HAI 2.0/testing/test4.csv')
TEST_DF_RAW = pd.concat([test1, test2, test3, test4], axis=0)
TEST_DF_RAW

In [ ]:
TEST_DF = normalize(TEST_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET]).ewm(alpha=0.9).mean()
TEST_DF

In [ ]:
HAI_DATASET_TEST = HaiDataset(TEST_DF_RAW[TIMESTAMP_FIELD], TEST_DF, stride=1, attacks=None) # stride=1로 모든 데이터를 인코딩 할 수 있게함.

In [ ]:
HAI_DATASET_TEST[0], HAI_DATASET_TEST[1]

In [ ]:
def inference(dataset, model, batch_size):
    dataloader = DataLoader(dataset, batch_size=batch_size)
    ts, dist, att = [], [], []
    with torch.no_grad(): # 해당 블록을 history 트래킹 하지 않겠다는 뜻. / 학습이 아닌 추론이므로 loss, optimzer grad, step 등등 없음.
        for batch in dataloader: # dataloader -> HAI_DATASET_TEST를 배치셋화 함. (batch_size, seq=1, n_tag = 80)
            given = batch["given"].cuda()
            answer = batch["answer"].cuda()
            guess = model(given) # given x값을 학습된 model에 넣어 y_hat 도출. 
            ts.append(np.array(batch["ts"]))
            dist.append(torch.abs(answer - guess).cpu().numpy()) # y와 y_hat 값의 차이
            try: # 실행할 코드, 현재는 batch에 'attack' 속성이 없음.. - 아마도??
                att.append(np.array(batch["attack"]))
            except: # 실행할 코드가 안될시, att에 batchsize 만큼의 0 넣기 (최종적으로 test data 수 만큼 0이 넣어짐.)
                att.append(np.zeros(batch_size))
            
    return (
        np.concatenate(ts), # CHECK_TS
        np.concatenate(dist), # CHECK_DIST
        np.concatenate(att), # CHECK_ATT
    )

In [ ]:
def put_labels(distance, threshold):
    xs = np.zeros_like(distance)# ANOMALY_SCORE = distance 행렬과 같은 shape의 0으로 이루어진 numpy 행렬 xs 만듦. (358448,)
    xs[distance > threshold] = 1 # distance행렬 각 원소 > threshold 이면, xs numy 행렬의 그 위치에 1.
    return xs (358448,)

In [ ]:
%%time
# 모델을 불러 온 이후에는 이 모델을 학습 할 껀지, 사용 할 껀지에 따라 각각 model.train(), model.eval() 둘 중에 하나를 사용.
MODEL.eval() 
CHECK_TS, CHECK_DIST, CHECK_ATT = inference(HAI_DATASET_TEST, MODEL, BATCH_SIZE)

In [ ]:
CHECK_TS, CHECK_DIST, CHECK_ATT

In [ ]:
ANOMALY_SCORE = np.mean(CHECK_DIST, axis=1) 
# test데이터에서 inference함수로 도출된 y_hat값과 y의 차이로 구해진 1seq 내 모든 n_tag(80개)의 평균 (열 평균이므로 axis=1)

In [ ]:
ANOMALY_SCORE.shape

In [ ]:
def check_graph(xs, att, piece=2, THRESHOLD=None):
    l = xs.shape[0] # 전체 ANOMALY_SCORE 데이터 수 = 358448
    chunk = l // piece # 전체 데이터 수 358448을 piece = 3로 나눈 값, 이 값이 그래프 위에서 아래로 3개 중 각각 xtick 범위가 됨. chunk=119482.xx
    fig, axs = plt.subplots(piece, figsize=(20, 4 * piece))
    for i in range(piece): # i=0,1,2 
        L = i * chunk # L = 0, 119482, 23xxxx
        R = min(L + chunk, l) # R = min(119482,358448), min(23xxxx,358448), min(36xxxx,358448)
        xticks = range(L, R) # R그래프 3개 X범위 -> L,R = 0 ~ 12000, 12000 ~ 24000, 24000 ~ 36000
        axs[i].plot(xticks, xs[L:R])
        if len(xs[L:R]) > 0:
            peak = max(xs[L:R])
            axs[i].plot(xticks, att[L:R] * peak * 0.3)
        if THRESHOLD!=None:
            axs[i].axhline(y=THRESHOLD, color='r') # THRESHOLD가 NONE이 아니면, 빨간색으로 y = THRESHOLD 선을 긋는다.
    plt.show()

In [ ]:
THRESHOLD = 0.045
check_graph(ANOMALY_SCORE, CHECK_ATT, piece=3, THRESHOLD=THRESHOLD)

In [ ]:
LABELS = put_labels(ANOMALY_SCORE, THRESHOLD) #(358448,)
LABELS, LABELS.shape

In [ ]:
LABELS.sum()

In [ ]:
def fill_blank(check_ts, labels, total_ts):
    def ts_generator():
        for t in total_ts:
            yield dateutil.parser.parse(t)

    def label_generator():
        for t, label in zip(check_ts, labels):
            yield dateutil.parser.parse(t), label

    g_ts = ts_generator()
    g_label = label_generator()
    final_labels = []

    try:
        current = next(g_ts)
        ts_label, label = next(g_label)
        while True:
            if current > ts_label:
                ts_label, label = next(g_label)
                continue
            elif current < ts_label:
                final_labels.append(0)
                current = next(g_ts)
                continue
            final_labels.append(label)
            current = next(g_ts)
            ts_label, label = next(g_label)
    except StopIteration:
        return np.array(final_labels, dtype=np.int8)

In [ ]:
%%time
FINAL_LABELS = fill_blank(CHECK_TS, LABELS, np.array(VALIDATION_DF_RAW[TIMESTAMP_FIELD]))
FINAL_LABELS.shape

Submit

In [ ]:
submission = pd.read_csv('/content/drive/My Drive/dataset/산업보안AI_data/HAI 2.0/sample_submission.csv')
submission.index = submission['time']
submission.loc[CHECK_TS,'attack'] = LABELS
submission

In [ ]:
submission.to_csv('/content/drive/My Drive/dataset/산업보안AI_data/HAI 2.0/sample_submission.csv', index=False)